# 列舉所有資產的riskiness r

In [1]:
import sympy
from pandas import Series,DataFrame
from scipy.optimize import fsolve
import math
import time
import pandas as pd
import numpy  as  np
import os

# 無風險利率
* 我們須扣除無風險利率，得到超額報酬率以計算Riskiness r
* 目前使用LIBOR [1個月利率](https://www.global-rates.com/interest-rates/libor/american-dollar/usd-libor-interest-rate-overnight.aspx)

In [2]:
rfrate = 2.45785 / 12 /100
rfrate

0.0020482083333333333

# 關於超額報酬的數量級:
* 將要帶入公式的數值調整到介在0-100間，目的是為了讓大部分的Riskiness r介在0-100間。

In [3]:
def f1(x,arr_returns):
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)

In [4]:
def get_riskiness_r(guess , arr_returns):
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

# 計算Riskiness R，並依照R值排序，傳回前十名的ETF

In [5]:
def get_top5_etf(df):
    global rfrate
    
    etf_list = list(df.columns)
    
    # 計算每支ETF的riskiness r
    etf_dic = {}
    for etf_name in etf_list:
        arr_returns = df[etf_name] - rfrate  
        guess = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic[etf_name] = get_riskiness_r(guess,arr_returns)
    
    # ETF依照riskiness r排序
    sorted_x = sorted(etf_dic.items(), key=lambda kv: kv[1])
    #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前5名
    top5 = sorted_x[:5]
    
    # 取前5的ETF名稱
    etf_top5 = [etf[0] for etf in top5]
    
    #前5的平均
    top5_average = df[etf_top5].mean(axis=1)
    
    return etf_top5 , top5_average

# 讀取全部ETF資料夾

In [6]:
def get_total_months(data_path):
    subdir = os.listdir(data_path)[0] #取第一個group的csv檔
    file_path = data_path + subdir + '/' + subdir + '_return.csv'
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        total_rows = df.shape[0]
        return total_rows

In [7]:
data_path = '../2014to2018RE_data/'
# 以程式取得資料總筆數
total_data = get_total_months(data_path)
# 或是寫死都可
#total_data = 61
# 指定用來計算指標的資料個數
window_size = 30
rolling_times = total_data - window_size

In [8]:
for i in range(rolling_times):
    print('=== Rolling #',i+1,'===')
    allgroup_top5_return = DataFrame()
    allgroup_top5_dict = dict()
    for subdir in os.listdir(data_path):
        subdir_path = os.path.join(data_path, subdir)
        if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
            file = subdir + '_return.csv'
            file_path = os.path.join(subdir_path, file)
            if os.path.isfile(file_path):
                df = pd.read_csv(file_path)
                df = df.iloc[i:i+30,2:]
                # 提出[前10名的名稱,前10名的平均各期報酬]
                [ top5_etf,allgroup_top5_return[subdir] ] = get_top5_etf(df)
                print('Group:',subdir)
                print('Top5:',top5_etf)
                print('================')
                allgroup_top5_dict[subdir] = top5_etf
                df = pd.read_csv(file_path)
                etf_date = df['Date']

=== Rolling # 1 ===


C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


Group: Alternatives ETF List (35)
Top5: ['FLAG', 'MOM', 'SIZ', 'BTAL', 'CHEP']
Group: Broad Asia ETF List (44)
Top5: ['VXUS', 'FPA', 'AXJV', 'FPXI', 'GEM']
Group: Commodity ETF List (125)
Top5: ['IAU', 'SGOL', 'UCO', 'BOIL', 'DTO']
Group: Consumer Discretionary Equity ETF List (31)
Top5: ['XLY', 'RXI', 'FDIS', 'RCD', 'VCR']
Group: Consumer Staples Equity ETF List (26)
Top5: ['KXI', 'PSCC', 'XLP', 'FSTA', 'VDC']
Group: Crude Oil ETF List (22)
Top5: ['UCO', 'DTO', 'SCO', 'SZO', 'USL']
Group: Currency ETF List (36)
Top5: ['USDU', 'FXE', 'DBV', 'FXA', 'FXS']
Group: Developed Asia Pacific ETF List (108)
Top5: ['IDOG', 'DOO', 'VPL', 'EWJ', 'FDT']
Group: Developed Markets ETF List 1 (100)
Top5: ['IDOG', 'VSS', 'VT', 'URTH', 'KXI']
Group: Developed Markets ETF List 2 (100)
Top5: ['EFZ', 'DOO', 'PIZ', 'ACIM', 'EXI']
Group: Developed Markets ETF List 3 (93)
Top5: ['CROP', 'FTAG', 'PEX', 'EDOM', 'FID']
Group: Emerging Asia Pacific ETF List (114)
Top5: ['YINN', 'INDL', 'FXP', 'EPHE', 'INCO']
Group

Group: Investment Grade Corporate ETF List (50)
Top5: ['ICSH', 'IGSB', 'GSY', 'SPLB', 'VCLT']
Group: Junk ETF List (43)
Top5: ['HYND', 'HYZD', 'HYHG', 'ANGL', 'GHYG']
Group: Materials Equity ETF List (62)
Top5: ['DUST', 'JNUG', 'RING', 'GDXJ', 'CROP']
Group: Municipal Bond ETF List (29)
Top5: ['SUB', 'SMMU', 'PVI', 'SHM', 'PZA']
Group: Preferred Stock ETF List (12)
Top5: ['PFXF', 'PSK', 'PGF', 'PFF', 'FPE']
Group: Real Estate ETF List (48)
Top5: ['ICF', 'RWR', 'WPS', 'FRI', 'SCHH']
Group: Target Maturity Date Corporate Bond ETF List (24)
Top5: ['IBCD', 'IBDC', 'IBCE', 'BSCM', 'BSCJ']
Group: Technology Equity ETF List (75)
Top5: ['IGM', 'TDIV', 'IXN', 'FTEC', 'VGT']
Group: Telecom Equity ETF List (10)
Top5: ['IYZ', 'FCOM', 'VOX', 'FONE', 'XTL']
Group: Total Bond Market ETF List (82)
Top5: ['ISTB', 'HOLD', 'NEAR', 'ULST', 'MINT']
Group: Treasuries ETF List (51)
Top5: ['VGSH', 'SHY', 'TUZ', 'SCHO', 'STPP']
Group: Utilities Equity ETF List (24)
Top5: ['GII', 'VPU', 'XLU', 'IGF', 'UPW']
Gro

Group: Currency ETF List (36)
Top5: ['USDU', 'FXE', 'DBV', 'FXA', 'FXS']
Group: Developed Asia Pacific ETF List (108)
Top5: ['IDOG', 'DOO', 'VPL', 'EWJ', 'FDT']
Group: Developed Markets ETF List 1 (100)
Top5: ['IDOG', 'VSS', 'VT', 'HEFA', 'URTH']
Group: Developed Markets ETF List 2 (100)
Top5: ['EFZ', 'TOLZ', 'DOO', 'IPKW', 'PIZ']
Group: Developed Markets ETF List 3 (93)
Top5: ['CROP', 'PEX', 'FTAG', 'EFU', 'EDOM']
Group: Emerging Asia Pacific ETF List (114)
Top5: ['YINN', 'INDL', 'FXP', 'CN', 'PEK']
Group: Emerging Markets ETF List (79)
Top5: ['EEV', 'EDZ', 'EDC', 'EMQQ', 'GEM']
Group: Energy Equity ETF List (80)
Top5: ['NLR', 'GASL', 'ERY', 'FCG', 'GRID']
Group: Financials Equity ETF List (51)
Top5: ['KIE', 'VQT', 'KBWP', 'UYG', 'FINZ']
Group: Global ETF List (72)
Top5: ['AADR', 'GDXJ', 'SOCL', 'IFV', 'FONE']
Group: Gold ETF List (17)
Top5: ['DGLD', 'UGLD', 'DZZ', 'GLL', 'DGZ']
Group: Healthcare Equity ETF List (49)
Top5: ['XPH', 'BIB', 'IHI', 'XHE', 'IHF']
Group: Industrials Equity 

...

Group: Municipal Bond ETF List (29)
Top5: ['MEAR', 'SHYD', 'ITM', 'PVI', 'PZA']
Group: Preferred Stock ETF List (12)
Top5: ['PSK', 'PGF', 'PFXF', 'PFF', 'FPE']
Group: Real Estate ETF List (48)
Top5: ['FREL', 'RWX', 'IFGL', 'IYR', 'WREI']
Group: Target Maturity Date Corporate Bond ETF List (24)
Top5: ['IBDP', 'IBDQ', 'BSCO', 'IBDO', 'IBDC']
Group: Technology Equity ETF List (75)
Top5: ['QQQ', 'IGM', 'SCID', 'FTEC', 'VGT']
Group: Telecom Equity ETF List (10)
Top5: ['IYZ', 'IXP', 'FCOM', 'VOX', 'FONE']
Group: Total Bond Market ETF List (82)
Top5: ['MINT', 'HOLD', 'ULST', 'RAVI', 'NEAR']
Group: Treasuries ETF List (51)
Top5: ['TUZ', 'SHY', 'VGSH', 'SCHO', 'TBF']
Group: Utilities Equity ETF List (24)
Top5: ['GII', 'JXI', 'FXU', 'VPU', 'IDU']
Group: Volatility ETF List (18)
Top5: ['ZIV', 'SVXY', 'UVXY', 'VIXY', 'TVIX']
=== Rolling # 22 ===
Group: Alternatives ETF List (35)
Top5: ['QLS', 'FLAG', 'MOM', 'VAMO', 'HTUS']
Group: Broad Asia ETF List (44)
Top5: ['AIA', 'AAXJ', 'GEM', 'IEMG', 'MOTI'

Group: Developed Markets ETF List 1 (100)
Top5: ['VSS', 'KXI', 'SDIV', 'GII', 'JXI']
Group: Developed Markets ETF List 2 (100)
Top5: ['EFZ', 'ISRA', 'IVAL', 'CRAK', 'LOWC']
Group: Developed Markets ETF List 3 (93)
Top5: ['HGI', 'FLM', 'HSCZ', 'TLDH', 'PAGG']
Group: Emerging Asia Pacific ETF List (114)
Top5: ['AIA', 'EDIV', 'DEM', 'ASEA', 'FEM']
Group: Emerging Markets ETF List (79)
Top5: ['BICK', 'EEB', 'PIE', 'EDIV', 'DVYE']
Group: Energy Equity ETF List (80)
Top5: ['CRAK', 'GUSH', 'GASL', 'DRIP', 'GRID']
Group: Financials Equity ETF List (51)
Top5: ['PSCF', 'IAK', 'VQT', 'PHDG', 'KIE']
Group: Global ETF List (72)
Top5: ['AADR', 'CRAK', 'EEM', 'IEMG', 'EEMS']
Group: Gold ETF List (17)
Top5: ['DGLD', 'UBG', 'IAU', 'SGOL', 'GLD']
Group: Healthcare Equity ETF List (49)
Top5: ['LABU', 'IBB', 'UBIO', 'BIB', 'BBC']
Group: Industrials Equity ETF List (37)
Top5: ['FXR', 'IEV', 'CGW', 'VIS', 'PHO']
Group: Investment Grade Corporate ETF List (50)
Top5: ['IGSB', 'GSY', 'ICSH', 'LKOR', 'SPLB']
Gr

Group: Telecom Equity ETF List (10)
Top5: ['IYZ', 'IXP', 'FCOM', 'VOX', 'FONE']
Group: Total Bond Market ETF List (82)
Top5: ['FTSD', 'AGZD', 'LDUR', 'ULST', 'RAVI']
Group: Treasuries ETF List (51)
Top5: ['TUZ', 'SHY', 'VGSH', 'SCHO', 'TBF']
Group: Utilities Equity ETF List (24)
Top5: ['GII', 'JXI', 'FXU', 'VPU', 'IDU']
Group: Volatility ETF List (18)
Top5: ['UVXY', 'VIXY', 'ZIV', 'SVXY', 'TVIX']
=== Rolling # 27 ===
Group: Alternatives ETF List (35)
Top5: ['HTUS', 'CHEP', 'MOM', 'RLY', 'DIVA']
Group: Broad Asia ETF List (44)
Top5: ['AIA', 'XSOE', 'FPA', 'AAXJ', 'GEM']
Group: Commodity ETF List (125)
Top5: ['DBA', 'RJZ', 'DJP', 'BCM', 'DBB']
Group: Consumer Discretionary Equity ETF List (31)
Top5: ['JHMC', 'WBIA', 'FXD', 'RTH', 'PSCD']
Group: Consumer Staples Equity ETF List (26)
Top5: ['KXI', 'PSCC', 'IYK', 'PBJ', 'RHS']
Group: Crude Oil ETF List (22)
Top5: ['DTO', 'SCO', 'UCO', 'DBO', 'USL']
Group: Currency ETF List (36)
Top5: ['FXCH', 'FXS', 'FXA', 'CEW', 'FXC']
Group: Developed Asi

Group: Energy Equity ETF List (80)
Top5: ['GEX', 'PBW', 'QCLN', 'GUSH', 'DRIP']
Group: Financials Equity ETF List (51)
Top5: ['JHMF', 'BIZD', 'FXO', 'PFI', 'PSCF']
Group: Global ETF List (72)
Top5: ['TLEH', 'IQDF', 'JDST', 'GDXS', 'DBAW']
Group: Gold ETF List (17)
Top5: ['DGZ', 'DZZ', 'DGLD', 'GLL', 'UGLD']
Group: Healthcare Equity ETF List (49)
Top5: ['XLV', 'IHF', 'IBB', 'LABU', 'UBIO']
Group: Industrials Equity ETF List (37)
Top5: ['XAR', 'ITA', 'PPA', 'PHO', 'IYJ']
Group: Investment Grade Corporate ETF List (50)
Top5: ['IGSB', 'GSY', 'ICSH', 'LKOR', 'SPLB']
Group: Junk ETF List (43)
Top5: ['UJB', 'CJNK', 'HYND', 'HYIH', 'ANGL']
Group: Materials Equity ETF List (62)
Top5: ['MOO', 'GUNR', 'SBM', 'GRES', 'JDST']
Group: Municipal Bond ETF List (29)
Top5: ['MEAR', 'SHYD', 'ITM', 'PZA', 'TFI']
Group: Preferred Stock ETF List (12)
Top5: ['IPFF', 'PSK', 'PGF', 'PFXF', 'PFF']
Group: Real Estate ETF List (48)
Top5: ['FREL', 'RWX', 'IFGL', 'IYR', 'WREI']
Group: Target Maturity Date Corporate 

Group: Volatility ETF List (18)
Top5: ['UVXY', 'ZIV', 'SVXY', 'TVIX', 'VIIX']


In [ ]:
allgroup_top5_return = pd.concat([etf_date,allgroup_top5_return],axis=1)

In [ ]:
allgroup_top5_return.head()

# 將allgroup_top5 先存取成CSV

In [ ]:
allgroup_top5_return.to_csv("allgroup_top5_return_new.csv")

# 再次計算Riskiness R，並依照R值排序，傳回前五名的Group

In [ ]:
def get_top5_Group(data_file):

    df1 = pd.read_csv(data_file)
    etf_list1 = list(df1.columns)[2:] #<----------------------------------------------加上日期
    df1 = df1.iloc[:,2:]

    etf_dic1 = {}
    for etf_name in etf_list1:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        
        
        #-------------------------------------------------------------------------------
        
        #同上要加上時間
        
        
        arr_returns1 = df1[etf_name] - rfrate
        #arr_returns = arr_returns*1000      #調整數量級
        
        
        
        #---------------------------------------------------------------------------------
        
        guess1 = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic1[etf_name] = get_riskiness_r(guess1,arr_returns1)
    
    
    sorted_x1 = sorted(etf_dic1.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前5名
    top5 = sorted_x1[:5]
        # top5 = sorted_reversed_x[:5]
    #print(top5)
    
    # 取前5的ETF名稱
    etf_top5 = [etf[0] for etf in top5]

    df2 = df1[etf_top5]
    
    return df2

In [ ]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def penalty(w):
    if min(w)<0 or max(w)>1:
        return 100000
    else:
        return 0
def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))    

def bestweight(w):
    r = []
    r.append(penalty(w)+np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v+penalty(w)+np.abs(sum(w)-1))
    return r

In [ ]:
def fourdrop(x):
    a = []
    for i in range(len(x)):
        a.append(round(sum(x[i]),3))
    return a
x = np.array(allgroup_top5_return.iloc[:,[0]])
x = fourdrop(x)
y = np.array(allgroup_top5_return.iloc[:,[1]])
y = fourdrop(y)
all_return = pd.DataFrame([x,y]).T
all_return = all_return

In [ ]:
x = np.array([-0.01,0,0.0258])
y = np.array([-0.05,0,0.134])
all_return = pd.DataFrame([x,y]).T

# 依次計算32組的權重

In [ ]:
a = list(range(0, 32))
all_return = allgroup_top10_return.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

# 計算出group中 前五名的權重

In [ ]:
df2 = get_top5_Group("C:\\Users\\USER\\github\\HW1--data-mining\\FinalProject\\Scripts\\allgroup_top5_return_new.csv")

In [ ]:
df2.head()

In [ ]:
a = list(range(0, 5))
all_return = df2.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

In [ ]:
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)